In [36]:
import pandas as pd
import os

In [37]:
"""Run this cell to add a new gene to the MutPred predictions file
Input file\s should be a csv file of MutPred predictions for a single gene."""

# Change the path or file name!!! 
genes = ["GJB2"]
all_mutpred_files = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\MutPred2\\all_predictions.csv", header=None)

for gene in genes:
    # create list of files in the directory, that start with name of gene
    files = [f for f in os.listdir("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\MutPred2\\files") if f.startswith(gene)]
    for f in files:
        # create the path of the new gene
        new_gene_path = f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\MutPred2\\files\\{f}"
        # read the file of the new gene
        new_gene_predictions = pd.read_csv(new_gene_path, header=None)

        # add the new gene to the all_eve_files dataframe, without the header
        all_mutpred_files = pd.concat([all_mutpred_files, new_gene_predictions], ignore_index=True, axis=0)

# Sanity check that it includes all genes
column_names = ["gene", "variant", "MutPred_score", "explain", "MutPred_pred", "nothing"]
all_mutpred_files.columns = column_names
print(all_mutpred_files["gene"].unique())

# Save the new dataframe to a csv file
all_mutpred_files.to_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\MutPred2\\all_predictions.csv", index=False)

['gene' 'FGFR1_HUMAN' 'COL4A5_HUMAN' 'MYO7A_HUMAN' 'SLC26A4_HUMAN'
 'COL2A1_HUMAN' 'COL4A3_HUMAN' 'WFS1_HUMAN' 'GJB2_HUMAN']


In [38]:
# Load MutPred2 predictions
mutpred_predictions = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\MutPred2\\all_predictions.csv", header=None)
mutpred_predictions

,0,1,2,3,4,5
0,gene,variant,MutPred_score,explain,MutPred_pred,nothing
1,gene,variant,MutPred_score,explain,MutPred_pred,nothing
2,FGFR1_HUMAN,F9S,0.828,Altered Disordered interface (Pr = 0.25 | P = ...,ELME000041|ELME000106|ELME000137|ELME000146,-
3,FGFR1_HUMAN,W10P,0.922,Altered Ordered interface (Pr = 0.24 | P = 0.0...,ELME000041|ELME000137,-
4,FGFR1_HUMAN,A11R,0.872,Altered Disordered interface (Pr = 0.37 | P = ...,ELME000041|ELME000106,-
...,...,...,...,...,...,...
5051,GJB2_HUMAN,Y217F,0.092,-,-,-
5052,GJB2_HUMAN,G220A,0.101,-,-,-
5053,GJB2_HUMAN,G220K,0.142,-,-,-
5054,GJB2_HUMAN,K221R,0.08,-,-,-


In [39]:
# add headers
column_names = ["gene", "variant", "MutPred_score", "explain", "MutPred_pred", "nothing"]
mutpred_predictions.columns = column_names
mutpred_predictions = mutpred_predictions.drop(columns=["nothing", "explain", "MutPred_pred"])
mutpred_predictions

,gene,variant,MutPred_score
0,gene,variant,MutPred_score
1,gene,variant,MutPred_score
2,FGFR1_HUMAN,F9S,0.828
3,FGFR1_HUMAN,W10P,0.922
4,FGFR1_HUMAN,A11R,0.872
...,...,...,...
5051,GJB2_HUMAN,Y217F,0.092
5052,GJB2_HUMAN,G220A,0.101
5053,GJB2_HUMAN,G220K,0.142
5054,GJB2_HUMAN,K221R,0.08


In [40]:
# Delete the _HUMAN from the gene name
mutpred_predictions["gene"] = mutpred_predictions["gene"].str.replace("_HUMAN", "")
print(len(mutpred_predictions))

5056


In [41]:
# remove duplicates
mutpred_predictions = mutpred_predictions.drop_duplicates(subset=["gene", "variant"], keep="first")
print(len(mutpred_predictions))

4763


In [42]:
# Remove VUS variants
mutpred_predictions["MutPred_score"] = pd.to_numeric(mutpred_predictions['MutPred_score'], errors='coerce')
# Remove rows with revel score: 0.29 < revel score < 0.644
mutpred_VUS = mutpred_predictions[(mutpred_predictions["MutPred_score"] < 0.737) & (mutpred_predictions["MutPred_score"] >= 0.391)]
mutpred_predictions = mutpred_predictions[(mutpred_predictions["MutPred_score"] >= 0.737) | (mutpred_predictions["MutPred_score"] <= 0.391)]
mutpred_predictions

C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_28548\3920941166.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mutpred_predictions["MutPred_score"] = pd.to_numeric(mutpred_predictions['MutPred_score'], errors='coerce')


,gene,variant,MutPred_score
2,FGFR1,F9S,0.828
3,FGFR1,W10P,0.922
4,FGFR1,A11R,0.872
5,FGFR1,V12S,0.881
6,FGFR1,V14F,0.895
...,...,...,...
4778,GJB2,Y217F,0.092
4779,GJB2,G220A,0.101
4780,GJB2,G220K,0.142
4781,GJB2,K221R,0.080


In [43]:
# Add benign or pathogenic label to each variant, according to the VEST4 score.
mutpred_predictions["MutPred_pathogenicity"] = mutpred_predictions["MutPred_score"].apply(lambda x: 1 if x >= 0.764 else 0)
mutpred_predictions

,gene,variant,MutPred_score,MutPred_pathogenicity
2,FGFR1,F9S,0.828,1
3,FGFR1,W10P,0.922,1
4,FGFR1,A11R,0.872,1
5,FGFR1,V12S,0.881,1
6,FGFR1,V14F,0.895,1
...,...,...,...,...
4778,GJB2,Y217F,0.092,0
4779,GJB2,G220A,0.101,0
4780,GJB2,G220K,0.142,0
4781,GJB2,K221R,0.080,0


In [44]:
# print how many pathogenic and benign variants are in the EVE file
print(f"Number of pathogenic variants in MutPred: {len(mutpred_predictions[mutpred_predictions['MutPred_pathogenicity'] == 1])}")
print(f"Number of benign variants in MutPred: {len(mutpred_predictions[mutpred_predictions['MutPred_pathogenicity'] == 0])}")

Number of pathogenic variants in MutPred: 1824
Number of benign variants in MutPred: 1908


In [45]:
print(f"length of revel without VUS {len(mutpred_predictions)}")
print(f"length of revel VUS {len(mutpred_VUS)}")

length of revel without VUS 3732
length of revel VUS 1031


In [46]:
# Combine the prediction files into one file
my_prediction = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\predictions_vs_real_with_variant_all_genes_updated_221123.csv")
my_prediction

,variant,prediction,reality,gene
0,M1R,0,0,COL2A1
1,I2A,0,0,COL2A1
2,I2L,0,0,COL2A1
3,I2P,0,0,COL2A1
4,I2T,0,0,COL2A1
...,...,...,...,...
4799,Y217F,0,0,GJB2
4800,G220A,0,0,GJB2
4801,G220K,0,0,GJB2
4802,K221R,0,0,GJB2


In [47]:
# Merge the two files #TODO: Make sure it's done correctly
merged = pd.merge(my_prediction, mutpred_predictions, on=["gene", "variant"])
merged

,variant,prediction,reality,gene,MutPred_score,MutPred_pathogenicity
0,M1R,0,0,COL2A1,0.960,1
1,I2L,0,0,COL2A1,0.319,0
2,I2V,0,0,COL2A1,0.240,0
3,L4F,0,0,COL2A1,0.326,0
4,A6T,0,0,COL2A1,0.198,0
...,...,...,...,...,...,...
3766,Y217F,0,0,GJB2,0.092,0
3767,G220A,0,0,GJB2,0.101,0
3768,G220K,0,0,GJB2,0.142,0
3769,K221R,0,0,GJB2,0.080,0


In [48]:
# To make sure that both tools will be tested on the same variants, save the merged with REVEL file and do so for all other tools. Then merge all the files together.
merged_only_tool_columns = merged[['gene', 'variant', 'MutPred_score', 'MutPred_pathogenicity']]
merged_only_tool_columns.to_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\MutPred_on_dvd_data_predictions_LOPO.csv")

In [49]:
print(f"length of merged {len(merged)}")
print(f"length of my_prediction {len(my_prediction)}")
print(f"length of mutpred {len(mutpred_predictions)}")

length of merged 3771
length of my_prediction 4804
length of mutpred 3732


In [50]:
#############################################################################################################
# All next rows were used to calculate the MCC, before realised I must apply first the threshold and remove the VUS variants. Now all these rows aren't in use, since I'm doing the calculations after removing the VUS variants and with all tools together.
#############################################################################################################

In [51]:
# # Save the merged file to csv
# merged.to_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\merged_my_prediction_and_mutpred.csv", index=False)

In [52]:
## Calculate the MCC for the mutpred predictions, according to the real pathogenicity (0 or 1)
from sklearn.metrics import matthews_corrcoef

In [53]:
# separate the merged dataframe according to gene
genes = merged["gene"].unique()
print(f"Number of genes: {len(genes)}")

Number of genes: 8


In [54]:
# # Calculate MCC for each gene specific predictor for mutpred
# 
# # Build dictionary with gene names as keys.
# mccs = {gene: 0 for gene in genes}
# 
# for gene in genes:
#     gene_df = merged[merged["gene"] == gene]
#     # Assuming you have a DataFrame called 'data' with 'prediction' and 'MutPred_score' columns
#     # Create binary predictions based on the 0.5 threshold
#     gene_df.loc[:, 'binary_prediction_mutpred'] = (gene_df['MutPred_score'] > 0.5).astype(int)
#     
#     gene_df['predictions'] = gene_df['predictions'].astype(int)
#     
#     # Calculate MCC
#     mcc = matthews_corrcoef(gene_df['predictions'], gene_df['binary_prediction_mutpred'])
#     
#     # Get gene name for the use for the dictionary
#     gene = gene_df['gene'].unique()[0]
#     
#     # Append mcc to dictionary
#     mccs[gene] = mcc
# 
# print("MCCs of MutPred2 predictions for each gene:")
# for gene in mccs:
#     print(f"{gene}: {mccs[gene]}")


In [55]:
# from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, average_precision_score, f1_score, log_loss, confusion_matrix
# # store the results in a dictionary
# results = {gene: 0 for gene in genes}
# 
# for gene in genes:
#     gene_df = merged[merged["gene"] == gene]
#     y_true = gene_df["pathogenicity"].astype(int)
#     predicted_probabilities = gene_df["MutPred_score"]
#     gene_df.loc[:, 'binary_prediction_mutpred'] = (gene_df['MutPred_score'] > 0.5).astype(int)
#     predicted_labels = gene_df["binary_prediction_mutpred"]
#     
#     results_gene = {}
#     # Calculate AUC-ROC
#     roc_auc = roc_auc_score(y_true, predicted_probabilities)
#     # append to dictionary
#     results_gene["roc_auc"] = roc_auc
#     
#     # Calculate ROC curve
#     fpr, tpr, thresholds = roc_curve(y_true, predicted_probabilities)
#     roc_auc = auc(fpr, tpr)
#     # append to dictionary
#     results_gene["auc"] = roc_auc
#     
#     # Calculate AUC-PR
#     precision, recall, _ = precision_recall_curve(y_true, predicted_probabilities)
#     pr_auc = auc(recall, precision)
#     # append to dictionary
#     results_gene["pr_auc"] = pr_auc
#     
#     # Calculate F1 Score
#     f1 = f1_score(y_true, predicted_labels)
#     # append to dictionary
#     results_gene["f1"] = f1
#     
#     # Calculate Log Loss
#     logloss = log_loss(y_true, predicted_probabilities)
#     # append to dictionary
#     results_gene["logloss"] = logloss
#     
#     # Calculate confusion matrix
#     conf_matrix = confusion_matrix(y_true, predicted_labels)
#     # append to dictionary
#     results_gene["confusion_matrix"] = conf_matrix
#     
#     # Get gene name for the use for the dictionary
#     gene = gene_df['gene'].unique()[0]
#     # Append results to dictionary
#     results[gene] = results_gene
#     
# ## Print results
# print("Results of MutPred2 predictions for each gene:")
# for gene in results:
#     print(f"{gene}: {results[gene]}")

In [56]:
# # present the confusion matrix for each gene
# import matplotlib.pyplot as plt
# 
# for gene in results:
#     # Plot confusion matrix
#     plt.imshow(results[gene]["confusion_matrix"], cmap=plt.cm.Blues)
#     plt.xlabel("Predicted labels")
#     plt.ylabel("True labels")
#     plt.xticks([0, 1], ["Benign", "Pathogenic"])
#     plt.yticks([0, 1], ["Benign", "Pathogenic"])
#     plt.title(f"Confusion matrix for {gene}, MutPred2 predictions")
#     plt.style.use("seaborn-white")
#     # Add text annotations
#     for i in range(2):
#         for j in range(2):
#             plt.text(j, i, results[gene]["confusion_matrix"][i, j], ha="center", va="center", color="black", backgroundcolor="white")
#     plt.colorbar()
#     plt.show()

In [57]:
# # present the ROC curve for each gene
# for gene in results:
#     # Plot ROC curve
#     plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (area = {results[gene]['roc_auc']:.2f})")
#     plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel("False Positive Rate")
#     plt.ylabel("True Positive Rate")
#     plt.title(f"ROC curve for {gene}, MutPred2 predictions")
#     plt.legend(loc="lower right")
#     plt.show()